<a href="https://colab.research.google.com/github/Sylver640/ADA-Informes/blob/main/Informe_BellmanFord.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Descripción del problema (camino más corto)
---
**Entrada**: Un grafo dirigido $G=(V,E)$, un vértice fuente $s\in V$, y un valor real $l_e \geq 0$ asociado a cada arco $e\in E$.

**Salida**: La distancia más corta $dist(s,v)$ para cada vértice $v\in V$.

----

Continuando la linea de problemas clásicos en la teoría de grafos, era imposible dejar fuera al conocido **camino más corto**, en este caso con una fuente única. En términos generales, esta versión del problema tiene como objetivo encontrar la **distancia más corta** entre un vértice inicial o fuente $s$ y todos los nodos del grafo dirigido $G = (V, E)$. Definiremos la distancia o largo de un camino como la suma de sus arcos. 

Por ejemplo, en el siguiente grafo, las distancias más cortas entre $s$ y el resto de los nodos son: $distancia(s,s)=0$, $distancia(s,v)=1$, $distancia(s,w)=3$ y $distancia(s,t)=6$.

![image](https://chartreuse-goal-d5c.notion.site/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F9353642d-b383-4a2c-96b0-479eb5240879%2FUntitled.png?table=block&id=0491d913-c844-4de0-a5a4-9b1bfe386320&spaceId=4f8bebe4-a843-44d2-b6ee-51e2006a90d1&width=2000&userId=&cache=v2)

Un algoritmo que logra entregar una solución óptima a esta problemática es el conocido **algoritmo de Dijkstra**, el cual es capaz de encontrar la distancia más corta en grafos con solamente arcos positivos. 

No obstante, esta problemática no se limita a tan solo aplicaciones que sean de esta índole. Por lo tanto, una versión revisada del problema que permita esto tendría las siguientes características:

---

**Entrada**: Un grafo dirigido $G=(V,E)$, un vértice fuente $s\in V$, y un valor real $l_e$ asociado a cada arco $e\in E$.

**Salida**: Una de las siguientes opciones:

1. La distancia más corta $dist(s,v)$ para cada vértice $v\in V$.
2. Una declaración indicando que $G$ contiene un ciclo negativo.

---

Así tenemos el mismo problema, el cual nuevamente tiene el objetivo de encontrar el camino más corto desde una fuente única. Esto sobre grafos dirigidos y con arcos **positivos o negativos**. Sin embargo, con algo más de versatilidad tenemos una limitante, y estos son los llamados **ciclos negativos**, los cuales impiden entregar una correcta solución al problema, puesto que podríamos iterar indefinidamente en este ciclo reduciendo infinitamente el largo del camino. En esta versión del problema, un algoritmo capaz de solucionarlo es el **algoritmo de Bellman-Ford**, el cual trabaja bajo el paradigma de la programación dinámica.

#2. Algoritmos
A continuación, se presentarán las implementaciones de los dos algoritmos mencionados anteriormente: Dijkstra y Bellman-Ford. Para poder trabajar con ellos, se utilizará el generador de instancias propuesto en el siguiente bloque de código.

In [ ]:
import random

### Generadores de instancia ###

def is_valid_edge(generated_edges: dict, i: int, j: int):
    return i != j and not generated_edges.get((i, j), None) and not generated_edges.get((j, i), None)

def instance_generator_bellman(n: int):
    """
        Input: cantidad de vértices
        Output: una lista que contiene todos los arcos y el número del vértice fuente (la función retorna dos variables).
        Los arcos vienen en la forma (i, j, weight), donde i es el vértice origen del arco y j el vértice al que apunta el arco, mientras que weight es su peso.
    """
    graph = []
    nodes = random.sample(range(0, n), n)
    unvisited_nodes = random.sample(range(0, n), n)
    
    generated_edges = {}
    for i in nodes:
        rand = random.sample(nodes, random.randint(1, 3))

        for j in rand:
            edge = (i, j)
            edge_with_weight = (i, j, random.randint(1, 100))
            
            if generated_edges.get((edge[1], edge[0]), None):
                continue
            
            if i == j:
                new_vertice = None
                iterations = 0
                while new_vertice is None and iterations < 250:
                    iterations += 1
                    number = random.randint(0, n - 1)
                    if is_valid_edge(generated_edges, i, number):
                        new_vertice = number

                if iterations >= 250:
                    return instance_generator_bellman(n)
                
                edge = (i, new_vertice)
                edge_with_weight = (i, new_vertice, random.randint(-25, 100)) # -25 y 100 corresponde a los límites de los pesos, puede cambiarlos.
            
            graph.append(edge_with_weight)
            generated_edges[edge] = edge

            if edge_with_weight[1] in unvisited_nodes:
                unvisited_nodes.remove(edge_with_weight[1])

    for i in unvisited_nodes:
        valid_edge = False
        iterations = 0
        while not valid_edge and iterations < 250:
            iterations += 1
            m = random.randint(0, n - 1)
            if is_valid_edge(generated_edges, m, i):
                valid_edge = True
                edge = (m, i)
                edge_with_weight = (m, i, random.randint(-25, 100)) # -25 y 100 corresponde a los límites de los pesos, puede cambiarlos.
                graph.append(edge_with_weight)
                generated_edges[edge] = edge

        if iterations >= 250:
            return instance_generator_bellman(n)

    return graph, graph[0][0]

def instance_generator_dijkstra(n: int):
    """
        Input: cantidad de vértices
        Output: una lista que contiene todos los arcos y el número del vértice fuente (la función retorna dos variables).
        Los arcos vienen en la forma (i, j, weight), donde i es el vértice origen del arco y j el vértice al que apunta el arco, mientras que weight es su peso.
    """
    graph = []
    nodes = random.sample(range(0, n), n)
    unvisited_nodes = random.sample(range(0, n), n)
    
    generated_edges = {}
    for i in nodes:
        rand = random.sample(nodes, random.randint(1, 3))

        for j in rand:
            edge = (i, j)
            edge_with_weight = (i, j, random.randint(1, 100))
            
            if generated_edges.get((edge[1], edge[0]), None):
                continue
            
            if i == j:
                new_vertice = None
                iterations = 0
                while new_vertice is None and iterations < 250:
                    iterations += 1
                    number = random.randint(0, n - 1)
                    if is_valid_edge(generated_edges, i, number):
                        new_vertice = number

                if iterations >= 250:
                    return instance_generator_dijkstra(n)
                
                edge = (i, new_vertice)
                edge_with_weight = (i, new_vertice, random.randint(0, 100)) # -25 y 100 corresponde a los límites de los pesos, puede cambiarlos.
            
            graph.append(edge_with_weight)
            generated_edges[edge] = edge

            if edge_with_weight[1] in unvisited_nodes:
                unvisited_nodes.remove(edge_with_weight[1])

    for i in unvisited_nodes:
        valid_edge = False
        iterations = 0
        while not valid_edge and iterations < 250:
            iterations += 1
            m = random.randint(0, n - 1)
            if is_valid_edge(generated_edges, m, i):
                valid_edge = True
                edge = (m, i)
                edge_with_weight = (m, i, random.randint(0, 100)) # -25 y 100 corresponde a los límites de los pesos, puede cambiarlos.
                graph.append(edge_with_weight)
                generated_edges[edge] = edge

        if iterations >= 250:
            return instance_generator_dijkstra(n)

    return graph, graph[0][0]

##2.1 Algoritmo de Dijkstra
También llamado "algoritmo de caminos mínimos", éste fue concebido por el científico en computación Edsger Dijkstra en 1956, y publicado en 1959. Soluciona la primera versión del problema, es decir, solo trabaja con arcos positivos.

In [ ]:
from termcolor import cprint

def dijkstraAlgorithm(G, s, verbose, visualize):
  distancias = [1e7] * len(G)
  distancias[s] = 0
  
  return 0

#Ejemplo
n_dijkstra = random.randint(5,25)
G_dijkstra, fuente_dijkstra = instance_generator_dijkstra(n_dijkstra)

print(f"Dijkstra: {G_dijkstra}")
print(f"Fuente: {fuente_dijkstra}")

dijkstraAlgorithm(G_dijkstra, fuente_dijkstra, verbose = False, visualize = False)

Dijkstra: [(7, 9, 41), (7, 10, 1), (7, 18, 77), (4, 14, 21), (4, 13, 89), (4, 12, 51), (11, 2, 99), (11, 5, 3), (1, 10, 80), (1, 11, 13), (5, 18, 27), (18, 17, 43), (19, 4, 91), (15, 6, 46), (15, 11, 4), (15, 13, 81), (3, 2, 97), (3, 17, 80), (8, 15, 17), (0, 10, 8), (10, 6, 50), (6, 16, 2), (6, 3, 17), (14, 8, 49), (14, 1, 53), (16, 14, 11), (9, 18, 99), (9, 12, 79), (9, 8, 47), (12, 19, 45), (12, 20, 21), (20, 15, 50), (17, 11, 28), (17, 13, 20), (2, 5, 51), (13, 20, 9), (3, 7, 88), (4, 0, 88)]
Fuente: 7


0

###2.1.1 Descripción del algoritmo
Éste recibe un grafo dirigido de arcos positivos $G = (V, E)$ y un vértice fuente $s \in V$, y retorna una lista con la distancia más corta entre $s$ y cada uno de los demás vértices del grafo. Cabe notar que este algoritmo posee un enfoque **greedy**, en el sentido que siempre buscará la mejor solución en cada iteración, con la esperanza de que este camino sea el más corto para todo el problema. En otras palabras, Dijkstra construye una solución paso a paso en base a una regla heurística. En términos generales, el algoritmo funciona de la siguiente manera:
1. Creamos una lista vacía que almacenará cada distancia entre el vértice fuente y los demás vértices. Inicializamos el valor $dist[s]$ como $0$, y las posiciones restantes como un valor muy grande, puesto que siempre querremos buscar la mínima distancia.

###2.1.2 Ejemplo

###2.1.3 Visualización del camino más corto encontrado

###2.1.4 Ejecución del algoritmo paso a paso (`verbose = True`)

##2.2 Algoritmo de Bellman-Ford

In [177]:
from termcolor import cprint

def imprimirDistancias(G, nodos, distancias):
  print("Distancia de cada vértice desde la fuente:")
  for i in range(len(distancias)):
    print("{0}\t\t{1}".format(i, distancias[i]))

def bellmanFordAlgorithm(G, s, nodos, verbose, visualize):
  distancia = [99999999] * nodos
  distancia[s] = 0

  for _ in range(nodos-1):
    for u, v, w in G:
      if distancia[u] != 99999999 and distancia[u] + w < distancia[v]:
        distancia[v] = distancia[u] + w
  
  for u, v, w in G:
    if distancia[u] != 99999999 and distancia[u] + w < distancia[v]:
      print("CICLO NEGATIVO!")
      return []

  return distancia

#Ejemplo
n_bellman = random.randint(5,25)
G_bellman, fuente_bellman = instance_generator_bellman(n_bellman)

cprint(f"Grafo de entrada: {G_bellman}", 'yellow', attrs=["bold"])
cprint(f"Número de nodos: {n_bellman}")
cprint(f"Fuente: {fuente_bellman}", "yellow", attrs=["bold"])

distancias = bellmanFordAlgorithm(G_bellman, fuente_bellman, n_bellman, verbose = False, visualize  = False)

if (distancias != []): imprimirDistancias(G_bellman, n_bellman, distancias)

Grafo de entrada: [(4, 5, 44), (4, 0, 17), (21, 7, 51), (21, 20, 96), (21, 18, 89), (10, 15, 96), (15, 14, 50), (15, 19, 3), (15, 24, 90), (22, 21, 28), (12, 23, 4), (12, 0, 84), (11, 20, 6), (11, 22, 59), (11, 19, 70), (7, 12, 16), (23, 4, 62), (14, 18, -15), (14, 8, 41), (9, 20, 24), (9, 19, 81), (17, 3, 51), (17, 11, 100), (18, 10, 68), (18, 13, 95), (16, 23, 59), (3, 1, 87), (6, 21, 68), (5, 7, 61), (1, 17, 5), (1, 11, 47), (20, 3, 9), (20, 23, 27), (20, 18, 61), (24, 17, 61), (2, 14, 92), (2, 3, 54), (2, 9, 62), (8, 22, 5), (8, 20, 95), (13, 19, 90), (19, 17, 53), (19, 14, 45), (19, 22, 14), (0, 1, 73), (0, 11, 73), (0, 15, 81), (17, 6, 22), (15, 16, 41), (5, 2, 63)]
Número de nodos: 25
Fuente: 4
Distancia de cada vértice desde la fuente:
0		17
1		90
2		107
3		105
4		0
5		44
6		117
7		105
8		187
9		169
10		199
11		90
12		121
13		226
14		146
15		98
16		139
17		95
18		131
19		101
20		96
21		143
22		115
23		123
24		188


###2.2.1 Descripción del algoritmo
Este algoritmo recibe un grafo dirigido $G = (V,E)$ que puede poseer arcos negativos y un vértice fuente $s \in V$, y tiene dos posibles retornos:
1. Si no se encuentran ciclos negativos en el grafo, se retorna una lista con la distancia más corta entre $s$ y cada uno de los demás vértices del grafo.
2. En el caso contrario, se retorna una lista vacía y se muestra un mensaje en pantalla indicando la presencia de un ciclo negativo en $G$.

Cabe notar que Bellman-Ford trabaja bajo el paradigma de **programación dinámica**, por lo tanto será necesario definir su subestructura óptima, y en consecuencia su función de recurrencia, lo cual hace posible su implementación bottom-up.

###**Subestructura óptima:**
En términos generales, deseamos encontrar la ruta óptima $P$ considerando $i$ arcos o menos, puesto que no sabemos la cantidad específica de arcos de la ruta óptima. Si definimos esto, pueden ocurrir dos situaciones:
1. La ruta más corta tiene $i-1$ arcos o menos, por lo que tan solo bastaría con encontrar ésta.
2. La ruta más corta posee efectivamente $i$ arcos, por lo que se realiza la subdivisión de problemas especificada en la siguiente imagen.
![image](https://chartreuse-goal-d5c.notion.site/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2Ffddd9b5c-071a-4413-8f7d-229ad407e880%2FUntitled.png?table=block&id=e9d843f8-abec-4773-925d-3e254110810d&spaceId=4f8bebe4-a843-44d2-b6ee-51e2006a90d1&width=2000&userId=&cache=v2)

Como se puede observar, la ruta más corta $P$ se puede obtener calculando las rutas más cortas $P'$ entre $s$ y un nodo intermedio $w$, limitada a $i-1$ arcos. Luego, tan solo sumamos la distancia entre $w$ y $v$, lo cual se espera nos deje con la mejor alternativa. Por lo tanto, y viendo como es que la cantidad máxima de arcos para una ruta más corta es de $n-1$, para obtener la mejor solución al problema original debemos encontrar la ruta más corta con $i = n-1$.

Así, básandonos en este análisis previo, llegamos al siguiente lema que definirá nuestra subestructura óptima:

*Sea $G=(V,E)$ un grafo dirigido con largo de arcos reales y una fuente $s \in V$. Suponiendo que $i \geq 1$ y $v\in V$, y sea $P$ la ruta más corta $s \leadsto v$ en $G$ con $i$ arcos o menos. Entonces, una de las dos afirmaciones siguientes es verdadera:*

- *$P$ es la ruta más corta con $i-1$ arcos o menos.*
- *$P$ es, para algún valor $w \in V$, la ruta más corta $s\leadsto w$ con $i-1$ arcos o menos, adicionada con el arco $(w,v) \in E$.*

Finalmente, como consecuencia de estos subproblemas, es posible definir nuestra **función de recurrencia**, donde consideramos a $L_{i,v}$ como el largo mínimo de un camino $s\leadsto v$ con a lo más $i-1$ arcos y ciclos permitidos. Luego, para todo $i\geq 1$ y $v \in V$ tenemos que:

$L_{i,v} =
\min \left\{
 \begin{array}{cc}
 L_{i-1,v} & \text{(caso 1)} \\
 \min\limits_{(w,v)\in E} \{L_{i-1,w}+l_{wv}\} & \text{(caso 2)}
    	\end{array}
\right\}$

Siendo los **subproblemas base**, para todo $v \in V$, los siguientes:
$L_{0,v} =
\left\{
\begin{array}{cc}
0 & \text{si $$s=v$$} \\
+\infty & \text{en otro caso.}
\end{array}
\right.$

###2.2.2 Ejemplo

###2.2.3 Visualización del camino más corto encontrado

###2.2.4 Ejecución del algoritmo paso a paso (`verbose = True`)

#3. Correctitud del algoritmo de Bellman-Ford

#4. Tiempo de ejecución

#5. Experimentos